<a href="https://colab.research.google.com/github/elaiinechenn/Python_Projects/blob/main/Parking_Lot_LightGBM_Model_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd

#安裝LightGBM
import lightgbm as lgb

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
df = pd.read_csv('/content/drive/MyDrive/公開資料夾/all_data (1).csv',encoding='utf-8')
df = df.drop(labels=['Unnamed: 0','is_normal'],axis='columns') 

# 轉換天氣資料型態

In [30]:
df = df.replace('...', np.nan)
df = df.replace('/', np.nan)
df = df.replace('X', np.nan)
df = df.replace('T', np.nan)

df

,id,Time,ObsTime,availablecar,updatetime,week,obs_id,StnPres,SeaPres,temperature,...,WD/WS.1,WD/WS.2,WD/WS.3,Precp,Precp.1,SunShine,SunShine.1,visibility,UVI,Cloud
0,1,2021-01-01,0,764.0,2021-01-01 00:06:00,4,信義,1019.7,NaN,10.9,...,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2021-01-01,0,764.0,2021-01-01 00:16:00,4,信義,1019.7,NaN,10.9,...,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2021-01-01,0,764.0,2021-01-01 00:26:00,4,信義,1019.7,NaN,10.9,...,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2021-01-01,0,764.0,2021-01-01 00:36:00,4,信義,1019.7,NaN,10.9,...,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2021-01-01,0,764.0,2021-01-01 00:46:00,4,信義,1019.7,NaN,10.9,...,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378208,619,2021-02-28,15,26.0,2021-02-28 15:04:00,6,天母,1012.0,NaN,20.1,...,149,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2378209,619,2021-02-28,15,27.0,2021-02-28 15:14:00,6,天母,1012.0,NaN,20.1,...,149,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2378210,619,2021-02-28,15,27.0,2021-02-28 15:24:00,6,天母,1012.0,NaN,20.1,...,149,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2378211,619,2021-02-28,15,28.0,2021-02-28 15:34:00,6,天母,1012.0,NaN,20.1,...,149,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df['StnPres'] = df['StnPres'].astype(float, errors = 'raise')
df['SeaPres'] = df['SeaPres'].astype(float, errors = 'raise')
df['temperature'] = df['temperature'].astype(float, errors = 'raise')
df['dew point'] = df['dew point'].astype(float, errors = 'raise')
df['RH'] = df['RH'].astype(float, errors = 'raise')
df['WD/WS'] = df['WD/WS'].astype(float, errors = 'raise')
df['WD/WS.1'] = df['WD/WS.1'].astype(float, errors = 'raise')
df['WD/WS.2'] = df['WD/WS.2'].astype(float, errors = 'raise')
df['WD/WS.3'] = df['WD/WS.3'].astype(float, errors = 'raise')
df['Precp'] = df['Precp'].astype(float, errors = 'raise')
df['Precp.1'] = df['Precp.1'].astype(float, errors = 'raise')
df['SunShine'] = df['SunShine'].astype(float, errors = 'raise')
df['SunShine.1'] = df['SunShine.1'].astype(float, errors = 'raise')
df['visibility'] = df['visibility'].astype(float, errors = 'raise')
df['UVI'] = df['UVI'].astype(float, errors = 'raise')
df['Cloud'] = df['Cloud'].astype(float, errors = 'raise')

In [32]:
df['updatetime'] = pd.to_datetime(df['updatetime'])

In [33]:
df['updatetime'] = df['updatetime'].dt.ceil('10min')    #調整時間分鐘為十分位數，6分直接變10分
start = df['updatetime'].min()
end = df['updatetime'].max()
index_date = pd.date_range(start, end, freq='10min')

def supplement(df):
    df = df.set_index(['updatetime'])
    df = df.reindex(index_date)
    df['availablecar'] = df['availablecar'].interpolate(method ='linear')
    return df.reset_index()

In [34]:
df['minute'] = df['updatetime'].dt.minute
df

,id,Time,ObsTime,availablecar,updatetime,week,obs_id,StnPres,SeaPres,temperature,...,WD/WS.2,WD/WS.3,Precp,Precp.1,SunShine,SunShine.1,visibility,UVI,Cloud,minute
0,1,2021-01-01,0,764.0,2021-01-01 00:10:00,4,信義,1019.7,NaN,10.9,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,10
1,1,2021-01-01,0,764.0,2021-01-01 00:20:00,4,信義,1019.7,NaN,10.9,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,20
2,1,2021-01-01,0,764.0,2021-01-01 00:30:00,4,信義,1019.7,NaN,10.9,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,30
3,1,2021-01-01,0,764.0,2021-01-01 00:40:00,4,信義,1019.7,NaN,10.9,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,40
4,1,2021-01-01,0,764.0,2021-01-01 00:50:00,4,信義,1019.7,NaN,10.9,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378208,619,2021-02-28,15,26.0,2021-02-28 15:10:00,6,天母,1012.0,NaN,20.1,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,10
2378209,619,2021-02-28,15,27.0,2021-02-28 15:20:00,6,天母,1012.0,NaN,20.1,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,20
2378210,619,2021-02-28,15,27.0,2021-02-28 15:30:00,6,天母,1012.0,NaN,20.1,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,30
2378211,619,2021-02-28,15,28.0,2021-02-28 15:40:00,6,天母,1012.0,NaN,20.1,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,40


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2378213 entries, 0 to 2378212
Data columns (total 24 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   Time          object        
 2   ObsTime       int64         
 3   availablecar  float64       
 4   updatetime    datetime64[ns]
 5   week          int64         
 6   obs_id        object        
 7   StnPres       float64       
 8   SeaPres       float64       
 9   temperature   float64       
 10  dew point     float64       
 11  RH            float64       
 12  WD/WS         float64       
 13  WD/WS.1       float64       
 14  WD/WS.2       float64       
 15  WD/WS.3       float64       
 16  Precp         float64       
 17  Precp.1       float64       
 18  SunShine      float64       
 19  SunShine.1    float64       
 20  visibility    float64       
 21  UVI           float64       
 22  Cloud         float64       
 23  minute        int64         
dty

# 預測

In [36]:
df = df.sort_values(['id', 'updatetime'])
df['after_1hour_availablecar'] = df['availablecar'].shift(-6)
df['after_10min_availablecar'] = df['availablecar'].shift(-1)

In [37]:
#for迴圈切分資料 id為變數 ,df.iloc[:int(len(df)* 4/5)]
train_l=[]
test_l=[]
for i in df['id'].unique():   
  df_tmp = df[df['id'] == i]
  X_train ,Y_train= df_tmp.iloc[:int(len(df_tmp)*(4/5))],df_tmp.iloc[int(len(df_tmp)*(4/5)):]
  train_l.append(X_train)
  test_l.append(Y_train)

In [38]:
df_train = pd.concat(train_l)
df_test = pd.concat(test_l)

In [39]:
df_train.columns

Index(['id', 'Time', 'ObsTime', 'availablecar', 'updatetime', 'week', 'obs_id',
       'StnPres', 'SeaPres', 'temperature', 'dew point', 'RH', 'WD/WS',
       'WD/WS.1', 'WD/WS.2', 'WD/WS.3', 'Precp', 'Precp.1', 'SunShine',
       'SunShine.1', 'visibility', 'UVI', 'Cloud', 'minute',
       'after_1hour_availablecar', 'after_10min_availablecar'],
      dtype='object')

In [40]:
feature=['ObsTime', 'availablecar', 'week',
       'StnPres', 'SeaPres', 'temperature', 'dew point', 'RH', 'WD/WS',
       'WD/WS.1', 'WD/WS.2', 'WD/WS.3', 'Precp', 'Precp.1', 'SunShine',
       'SunShine.1', 'visibility', 'UVI', 'Cloud', 'minute',
       'after_1hour_availablecar']
y = ['after_10min_availablecar'] 
gbm =lgb.LGBMRegressor(boosting_type='gbdt',
            objective='regression',
            num_leaves=50,max_depth=6,learning_rate=0.1,n_estimators=500,
            metric=['mae'])

gbm.fit(df_train[feature],df_train[y])
df_test['pred']= gbm.predict(df_test[feature])

In [41]:
df_test

,id,Time,ObsTime,availablecar,updatetime,week,obs_id,StnPres,SeaPres,temperature,...,Precp.1,SunShine,SunShine.1,visibility,UVI,Cloud,minute,after_1hour_availablecar,after_10min_availablecar,pred
6790,1,2021-02-16,19,908.0,2021-02-16 20:00:00,1,信義,1008.1,NaN,20.4,...,NaN,NaN,NaN,NaN,NaN,NaN,0,903.0,907.0,899.271538
6791,1,2021-02-16,20,907.0,2021-02-16 20:10:00,1,信義,1008.8,NaN,18.8,...,NaN,NaN,NaN,NaN,NaN,NaN,10,904.0,905.0,900.307108
6792,1,2021-02-16,20,905.0,2021-02-16 20:20:00,1,信義,1008.8,NaN,18.8,...,NaN,NaN,NaN,NaN,NaN,NaN,20,902.0,903.0,900.590560
6793,1,2021-02-16,20,903.0,2021-02-16 20:30:00,1,信義,1008.8,NaN,18.8,...,NaN,NaN,NaN,NaN,NaN,NaN,30,902.0,904.0,900.646632
6794,1,2021-02-16,20,904.0,2021-02-16 20:40:00,1,信義,1008.8,NaN,18.8,...,NaN,NaN,NaN,NaN,NaN,NaN,40,903.0,904.0,900.506460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378208,619,2021-02-28,15,26.0,2021-02-28 15:10:00,6,天母,1012.0,NaN,20.1,...,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,27.0,19.546376
2378209,619,2021-02-28,15,27.0,2021-02-28 15:20:00,6,天母,1012.0,NaN,20.1,...,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,27.0,20.986293
2378210,619,2021-02-28,15,27.0,2021-02-28 15:30:00,6,天母,1012.0,NaN,20.1,...,NaN,NaN,NaN,NaN,NaN,NaN,30,NaN,28.0,20.818987
2378211,619,2021-02-28,15,28.0,2021-02-28 15:40:00,6,天母,1012.0,NaN,20.1,...,NaN,NaN,NaN,NaN,NaN,NaN,40,NaN,28.0,20.957904


In [42]:
df_test[df_test['pred']<0] #availablecar 四捨五入

,id,Time,ObsTime,availablecar,updatetime,week,obs_id,StnPres,SeaPres,temperature,...,Precp.1,SunShine,SunShine.1,visibility,UVI,Cloud,minute,after_1hour_availablecar,after_10min_availablecar,pred
16195,2,2021-02-23,4,0.0,2021-02-23 05:00:00,1,信義,1008.2,NaN,18.5,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,-0.013168
28315,4,2021-02-17,19,0.0,2021-02-17 19:10:00,2,信義,1016.9,NaN,15.2,...,NaN,NaN,NaN,NaN,NaN,NaN,10,1.0,0.0,-0.003191
28316,4,2021-02-17,19,0.0,2021-02-17 19:20:00,2,信義,1016.9,NaN,15.2,...,NaN,NaN,NaN,NaN,NaN,NaN,20,0.0,0.0,-0.096816
28317,4,2021-02-17,19,0.0,2021-02-17 19:30:00,2,信義,1016.9,NaN,15.2,...,NaN,NaN,NaN,NaN,NaN,NaN,30,0.0,2.0,-0.159530
28745,4,2021-02-20,18,0.0,2021-02-20 19:00:00,5,信義,1011.0,NaN,18.8,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,1.0,-0.000210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280695,604,2021-01-15,19,0.0,2021-01-15 19:50:00,4,內湖,1012.4,NaN,17.9,...,NaN,NaN,NaN,NaN,NaN,NaN,50,0.0,0.0,-0.092583
2280696,604,2021-01-15,19,0.0,2021-01-15 20:00:00,4,內湖,1012.4,NaN,17.9,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,-0.057324
2377756,619,2021-02-25,11,0.0,2021-02-25 11:30:00,3,天母,1007.3,NaN,23.7,...,NaN,NaN,NaN,NaN,NaN,NaN,30,1.0,0.0,-0.014710
2377899,619,2021-02-26,11,0.0,2021-02-26 11:30:00,4,天母,1007.2,NaN,23.4,...,NaN,NaN,NaN,NaN,NaN,NaN,30,0.0,0.0,-0.010273


In [43]:
# id_50 = df_normal_max[df_normal_max['id']==50]
# p = figure(title='id_50兩個月的資料變化', plot_width=1000, plot_height=450, x_axis_type='datetime')
# p.circle(id_50['updatetime'], id_50['availablecar'], color="black")
# p.line(id_50['updatetime'], id_50['availablecar'], color="red")
# p.xaxis.axis_label = '時間'
# p.yaxis.axis_label = '剩餘空位'
# show(p)

In [44]:
df_train['StnPres'] = df_train['StnPres'].str.replace('/','')
df_train['StnPres'].str.replace('...','').astype(float)
# df_train['StnPres'].astype(float)

AttributeError: ignored

In [ ]:
df_train['StnPres'].iloc[963522:963622]